In [1]:
import matplotlib.pyplot as plt
import numpy as np
from skimage.segmentation import felzenszwalb, slic, quickshift, watershed
from skimage.segmentation import mark_boundaries
from skimage.util import img_as_ubyte
from sklearn.metrics import pairwise_distances
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Activation, BatchNormalization
from tensorflow.keras.layers import Conv2D, Dense, Dropout 
from tensorflow.keras.layers import Flatten, Input, MaxPooling2D
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.preprocessing import image

2021-12-12 10:45:01.273997: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


2021-12-12 10:45:03.381491: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-12-12 10:45:03.382457: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-12-12 10:45:03.452641: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:65:00.0 name: Quadro RTX 4000 computeCapability: 7.5
coreClock: 1.545GHz coreCount: 36 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 387.49GiB/s
2021-12-12 10:45:03.452687: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2021-12-12 10:45:03.454073: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2021-12-12 10:45:03.454164: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.10
2021-1

In [3]:
# download MNIST datasets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
assert x_train.shape == (60000, 28, 28)
assert x_test.shape == (10000, 28, 28)
assert y_train.shape == (60000,)
assert y_test.shape == (10000,)

#  reformat images for tensorflor and normalize to 0-1 range and convert to float
input_shape = (28, 28, 1)

x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
x_train = x_train / 255.0
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1)
x_test = x_test / 255.0

# one-hot encode labels
y_train_labels = y_train.copy()
y_test_labels = y_test.copy()

y_train = tf.one_hot(y_train.astype(np.int32), depth=10)
y_test = tf.one_hot(y_test.astype(np.int32), depth=10)

2021-12-12 10:45:05.779844: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-12-12 10:45:05.780851: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:65:00.0 name: Quadro RTX 4000 computeCapability: 7.5
coreClock: 1.545GHz coreCount: 36 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 387.49GiB/s
2021-12-12 10:45:05.780932: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2021-12-12 10:45:05.780960: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2021-12-12 10:45:05.780975: I tensorflow/stream_executor/plat

InternalError: CUDA runtime implicit initialization on GPU:0 failed. Status: out of memory

In [ ]:
def bayesian_LIME_fit(img, model, n_perturbations=100):
    """
    get a Gaussian distribution of interpretable weights in the non-informative prior case

    arguments:
        img: the actual image ---> 4D tensor for inceptionnet
        model: classifier of interest
        n_perturbations: the number of desired perturbed samples 
        
    returns:
        the mean and standard deviation over the regression coefficients of a given image
    """
    y_true = predict_image_class_output(img, model)
    
    segments = felzenszwalb(np.squeeze(img), scale = 1, sigma = 1.5)
    #segments = watershed(np.squeeze(img), markers = 15) # TODO, look into segments
    #segments = quickshift(img[0].astype(np.int64), ratio=0.2, kernel_size=4, max_dist=200)

    x_lime = []
    y_lime = []
    weights = []
    
    ## begin image-dependent steps
    for i in range(n_perturbations):
        im_pert, seg_bool = choose_segments(np.squeeze(img), segments)
        x_lime.append(seg_bool)
        weights.append(calculate_distance_function(seg_bool, kernel_width=1.0))
        class_likelihood = predict_image_class_likelihood(np.reshape(im_pert, (1,28,28,1)), model, y_true)
        y_lime.append(class_likelihood)
    ## end image-dependent steps
    X = np.array(x_lime)
    y = np.array(y_lime)[:,0]

    W = np.diag(weights)

    alpha, lam = empirical_bayes(X, y, W, lam_init=1e-1, alpha_init=1e-1, iterations=5000, eps=1e-3)

    n, m = X.shape
    I_m = np.identity(m)
    mu_0 = np.zeros(m) #non-informative case
    s_n = np.linalg.inv(lam*I_m + alpha* X.T @ W @ X)
    mu_n = s_n @ (lam*I_m @ mu_0 + alpha * X.T @ W @ y)
    
    return mu_n, s_n, segments

In [ ]:
def empirical_bayes(X, y, W, lam_init=1e-1, alpha_init=1e-1, iterations=5000, eps=1e-3):
    lam = lam_init
    alpha = alpha_init

    n, m = X.shape
    I_m = np.identity(m)
    mu_0 = np.zeros(m) 
    
    eigenvals_fixed, eigenvecs = np.linalg.eig(X.T @ X)
    eigenvals_fixed = eigenvals_fixed.real

    lam_alpha_matrix = np.zeros((iterations, 2))

    for iters in range(iterations):
        eigenvals = eigenvals_fixed * alpha

        weighted_sum = [w_i / (lam + w_i) for w_i in eigenvals]

        gamma = np.sum(weighted_sum)

        s_n = np.linalg.inv(lam* I_m + alpha*X.T @ W @ X)
        mu_n = s_n @ (lam*I_m @ mu_0 + alpha * X.T @ W @ y)
        #mu_n = alpha * S_N @ X.T @ y

        lam = gamma / (mu_n.T @ mu_n)

        row_wise_inner_products = np.array([mu_n.T @ X[i,:] for i in range(len(X))])
        alpha = (n - gamma) / np.sum((y  - row_wise_inner_products)**2)

        lam_alpha_matrix[iters, 0] = lam
        lam_alpha_matrix[iters, 1] = alpha

        if (abs(lam_alpha_matrix[iters, 0] - lam_alpha_matrix[iters-1, 0]) < eps) and (abs(lam_alpha_matrix[iters, 1] - lam_alpha_matrix[iters-1, 1]) < eps):
                break
    print(f'alpha: {alpha}')
    print(f'lambda: {lam}')
    return alpha, lam

In [ ]:
def predict_image_class_output(img, model):
    # returns the class ID of the most likely predicted class
    # not used below

    output = model.predict(img)[0]
    class_out = np.where(output==max(output))[0]
    return class_out

def predict_image_class_likelihood(img, model, y_true):
    # returns the likelihood of the ground truth class

    likelihood = model.predict(img)[0][y_true]
    return likelihood

In [ ]:
def calculate_distance_function(seg_idx_one_hot, kernel_width = 1e-1):

    original = np.ones(len(seg_idx_one_hot)).reshape(1,-1)
    perturbations = seg_idx_one_hot.reshape(1,-1)
    #shapes: cos_similarity expects shape (1,n) for n superpixels
    #cosine similarity
    distances = pairwise_distances(perturbations,original, metric='cosine').flatten()

    weights = np.sqrt(np.exp(-(distances**2)/(kernel_width**2)))

    return weights.item()

In [ ]:
def visualization(img, segments, mu_n, s_n):
    #prior_array = segments.copy()
    weights_array = segments.copy()
    variances_array = segments.copy()

    for i, seg_val in enumerate(np.unique(segments)):
        #prior_array = np.where(prior_array==seg_val, mu_0[i], prior_array)
        weights_array = np.where(weights_array==seg_val, mu_n[i], weights_array)
        variances_array = np.where(variances_array==seg_val, s_n[i][i], variances_array) # ok to do?
       
    fig, ax = plt.subplots(1, 3, figsize=(18, 3))
    
    #ax[0].imshow(img.squeeze(), cmap = 'gray', alpha = 0.8)
    #ax[0].imshow(prior_array, cmap = 'Blues', alpha = 0.4)
    #ax[0].set_title('Prior Output')
    #ax[0].axis('off')  
    
    ax[1].imshow(img.squeeze(), cmap = 'gray', alpha = 0.4)
    ax[1].imshow(weights_array, cmap = 'Blues', alpha = 0.8)
    ax[1].set_title('Mu Output')
    ax[1].axis('off')   

    ax[2].imshow(img.squeeze(), cmap = 'gray', alpha = 0.4)
    ax[2].imshow(variances_array, cmap = 'Reds', alpha = 0.8)
    ax[2].set_title('Variance Output')
    ax[2].axis('off')